In [2]:
import requests
import os
import json
from bs4 import BeautifulSoup
from urllib.parse import urlparse

import mysql.connector
import re

In [3]:
from pyLightspeed.lsretail import api as lsretail
from pyLightspeed.lsecom import api as lsecom

In [4]:

#establishing the connection
conn = mysql.connector.connect(
   user='jamie', password='W!neL0ver', host='127.0.0.1', database='vintagewine')

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [5]:
KEY_FILE = "D:\Development\.keys\lightspeed_keys.json"

with open(KEY_FILE) as f:
    keys = json.load(f)

store_data = {
            'account_id': keys["account_id"],
            'save_path': 'D:\\Development\\.keys\\'
            }

credentials = {
            'client_id': keys["client_id"],
            'client_secret': keys["client_secret"],
            'api_key': keys["api_key"],
            'api_secret': keys["api_secret"]
            }

In [35]:
lsr = lsretail.Connection(store_data, credentials)
lse = lsecom.Connection(store_data, credentials)

In [7]:
page_link = 'https://www.wine.com/product/ridge-pagani-ranch-zinfandel-2016/510360'
item_id = '994'
system_id = '210000000998'

image_path = 'D:\\Data\\'
# this is the url that we've already determined is safe and legal to scrape from.
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}

In [36]:
#we can look up the system ID

item = lsr.get("Item", rid=994)
item["Item"]["systemSku"]

ERROR:root:200: Unhandled Exception: don't know what to do:  {'Date': 'Mon, 27 Jul 2020 16:28:23 GMT', 'Content-Type': 'application/json', 'Content-Length': '456', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=d6f548f9c939e99ff72f310ba3f6479241595867302; expires=Wed, 26-Aug-20 16:28:22 GMT; path=/; domain=.lightspeedapp.com; HttpOnly; SameSite=Lax; Secure', 'x-frame-options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-LS-Acct-Id': '190211', 'X-LS-OAuth-Client-Id': '71082', 'X-LS-API-Bucket-Level': '1/60', 'X-LS-Shard-Id': '23', 'X-LS-API-Drip-Rate': '1', 'X-LS-Master-System': 'false', 'X-LS-Master-Account': 'false', 'X-LS-Master-Catalog': 'false', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'CF-Cache-Status': 'DYNAMIC', 'cf-request-id': '0432b3a3800000b8f9140e3200000001', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Server': 'cloudflare', 'CF-RAY': '5b97bbb26b0bb

'210000000998'

In [8]:

page_response = requests.get(page_link, headers=headers, timeout=5)
# here, we fetch the content from the url, using the requests library
page_content = BeautifulSoup(page_response.content, "html.parser")
#we use the html parser to parse the url content and store it in a variable.
textContent = []


In [9]:
def process_image(url, image_path, filename, item_id, description = "Image", ordering = 0):
    with open(image_path+filename, "wb") as f:
        f.write(requests.get(url).content)
    url = lsr.api_url+'Image.json'
    
    files = {'image': (filename, open(image_path + filename, 'rb'), 'image/jpeg')}
    payload = {'data': '{"description": "' + description + '", "ordering": ' + ordering +', "itemID": ' + item_id +'}'}
    r = requests.post(url, files=files, data=payload, headers=lsr.headers)
    print(r.text)

# Get Content
Get's the content from the page


In [10]:
description = page_content.find(class_='viewMoreModule_text')
description_text = page_content.find(class_='viewMoreModule_text').text

In [11]:
page_content.select('body > div.corePage.winePage > main > section.pipSecContent > section.pipSecContent_topSection > div.pipWineNotes > div > div.viewMoreModule_text')


[<div class="viewMoreModule_text">Black cherry, pepper and chaparral on the nose. Briary red fruit, well defined, chalky tannins and refreshing acidity on the palate. Long sensuous finish. <p>Since 1991, Ridge has made zinfandel from the Pagani vineyard on the western side of Sonoma Valley. The majority of the vineyard was planted ninety to one hundred twenty years ago; portions of it were replanted between 2013—2018. The vineyard is meticulously tended by a quality-conscious family who has worked the land for four generations. An average site would have been abandoned during the thirteen years of Prohibition, or during the Depression. Its long survival is proof of its merit.</p><p>Blend: 88% Zinfandel, 9% Petite Sirah, 3% Alicante Bouschet</p></div>]

In [12]:
reviews = page_content.select('div.pipProfessionalReviews_list')
reviews

[<div class="pipProfessionalReviews_list"><div class="pipProfessionalReviews_rating"><span class="wineRatings_initials">V</span> <span class="wineRatings_rating">94</span></div><div class="pipProfessionalReviews_authorName">Vinous</div><div class="pipProfessionalReviews_review"><div class="pipSecContent_copy">The 2016 Zinfandel Pagani Ranch is bright, focused and beautifully perfumed. Bright red cherry, blood orange, pomegranate, mint and rose petal notes are all lifted in this exquisite, finessed Zinfandel. Silky, medium in body and gracious, the 2016 is absolutely sublime. The oak is just a bit pronounced at this stage, but that should not be an issue over time.</div></div></div>,
 <div class="pipProfessionalReviews_list"><div class="pipProfessionalReviews_rating"><span class="wineRatings_initials">RP</span> <span class="wineRatings_rating">93</span></div><div class="pipProfessionalReviews_authorName">Robert Parker's Wine Advocate</div><div class="pipProfessionalReviews_review"><div 

In [13]:
reviews[1].select('span[class="wineRatings_initials"]')

[<span class="wineRatings_initials">RP</span>]

In [14]:
page_content.find(class_='pipWineNotes_copy viewMoreModule').text

'Black cherry, pepper and chaparral on the nose. Briary red fruit, well defined, chalky tannins and refreshing acidity on the palate. Long sensuous finish. Since 1991, Ridge has made zinfandel from the Pagani vineyard on the western side of Sonoma Valley. The majority of the vineyard was planted ninety to one hundred twenty years ago; portions of it were replanted between 2013—2018. The vineyard is meticulously tended by a quality-conscious family who has worked the land for four generations. An average site would have been abandoned during the thirteen years of Prohibition, or during the Depression. Its long survival is proof of its merit.Blend: 88% Zinfandel, 9% Petite Sirah, 3% Alicante Bouschet View More'

## Store Webscrape
I keep webscrape data in a seperate table, so I want to update these. These can also be used to help train record matching models since I actually know what the main wine is, so storing that info too.

In [15]:

if page_content.find(class_='icon icon-screwcap prodAttr_icon prodAttr_icon-screwcap'):
    closure = 'Screwcap'
else:
    closure = ''

if page_content.find(class_='icon icon-glass-red prodAttr_icon prodAttr_icon-redWine'):
    category = 'Red'
elif page_content.find(class_='icon icon-glass-white prodAttr_icon prodAttr_icon-whiteWine'):
    category = 'White'
elif page_content.find(class_='icon icon-champagne prodAttr_icon prodAttr_icon-champagne'):
    category = 'Sparkling'
elif page_content.find(class_='icon icon-glass-white prodAttr_icon prodAttr_icon-roseWine'):
    category = 'Rose'
else:
    category = ''

msrp = re.findall('[0-9.]+', page_content.find(class_='productPrice').text)[0].strip()

In [16]:
insert_webscrape = ("INSERT INTO webscrapes "
                    "(retail_systemID, is_match, best_match, _web_scraper_order, web_scraper_start_url, scrape_sourceID, sku, product, product_href, title, name, vintage, producer, brand, price, msrp, region, subregion, description, size, alcohol, closure, category, varietal) "
                    "VALUES (%(retail_systemID)s, %(is_match)s, %(best_match)s, %(_web_scraper_order)s, %(web_scraper_start_url)s, %(scrape_sourceID)s, %(sku)s, %(product)s, %(product_href)s, %(title)s, %(name)s, %(vintage)s, %(producer)s, %(brand)s, %(price)s, %(msrp)s, %(region)s, %(subregion)s, %(description)s, %(size)s, %(alcohol)s, %(closure)s, %(category)s, %(varietal)s)")

data = {
    'retail_systemID': system_id,
    'is_match': 1,
    'best_match': 1,
    '_web_scraper_order': system_id,
    'web_scraper_start_url': page_link,
    'scrape_sourceID': 7,
    'sku':  page_content.find(attrs={'name':'productID'})["content"],
    'product': page_content.find(class_='pipName').text,
    'product_href': page_link,
    'title': page_content.find(class_='pipName').text,
    'name':  re.findall('[A-z ]+', page_content.find(class_='pipName').text)[0].strip(),
    'vintage': re.findall('[0-9]+', page_content.find(class_='pipName').text)[0].strip(),
    'producer': page_content.find(class_='pipWinery_headlineLink').text,
    'brand': page_content.find(class_='pipWinery_headlineLink').text,
    'price': re.findall('[0-9.]+', page_content.find(class_='productPrice').text)[0].strip(),
    'msrp': msrp,
    'region': page_content.find(attrs={'name':'productVarietal'})["content"],
    'subregion':  page_content.find(attrs={'name':'productRegion'})["content"],
    'description':  page_content.find(class_='viewMoreModule_text').text,
    'size': page_content.find(class_='prodAlcoholVolume_text').text,
    'alcohol': page_content.find(class_='prodAlcoholPercent_percent').text,
    'closure':  closure,
    'category': category,
    'varietal': page_content.find(attrs={'name':'productVarietal'})["content"]  
}

cursor.execute(insert_webscrape, data)
conn.commit()

## Store Reviews and Build Description

We store reviews in the Reviews table, and use them to create an HTML description to post.

<p>Clos des Quatre Vents is a bright garnet color. Fresh aromas of black fruits, cherry stone, mild spices permeate the nose, while the same fruits and spices are ripe and fresh on the palate, with silky tannins.</p>
<h3>Tasting Notes</h3>
<p><strong>93 James Suckling<br /></strong>Dried cherries with some flowers on the nose, following through to a medium body with menthol, bark and dried tea. Subtle and complex. From 60-year-old vines.</p>
<p>91 Vinous<br />Vivid magenta. Deeply perfumed dark berry, cherry liqueur and licorice aromas are complemented by hints of pungent flowers and allspice. Sweet and fleshy on the palate, offering concentrated blackberry, cherry pie, violet pastille and succulent herb flavors that become spicier as the wine opens up. Plays richness off of elegance with a steady hand and finishes long and sweet, with gently firming tannins and repeating florality.</p>

In [17]:
item_content =""
item_content = '<p>' + page_content.find(class_='pipWineNotes_copy viewMoreModule').text + '</p><h3>Tasting Notes</h3>'

In [18]:
for review in reviews:
    # Writing the HTML is failing, I am sure because I need to escape some things, but it isn't really needed so I am skipping it.
    insert_review = ("INSERT INTO wine_reviews "
                "(source, initials, rating, review_detail, wine_itemID, wine_systemSKU, url) "
                "VALUES (%(source)s, %(initials)s, %(rating)s, %(review_detail)s, %(wine_itemID)s, %(wine_systemSKU)s, %(url)s)")
    data = {
        'source': review.select('div[class="pipProfessionalReviews_authorName"]')[0].text,
        'initials': review.select('span[class="wineRatings_initials"]')[0].text,
        'rating': int(review.select('span[class="wineRatings_rating"]')[0].text),
        'review_detail': review.select('div[class="pipProfessionalReviews_review"]')[0].text,
        'wine_itemID': int(item_id),
        'wine_systemSKU': system_id,
        'url': page_link
        
    }
    item_content = item_content + "<p><strong>" + review.select('span[class="wineRatings_rating"]')[0].text + " " + review.select('div[class="pipProfessionalReviews_authorName"]')[0].text + "</strong><br />"
    item_content = item_content + review.select('div[class="pipProfessionalReviews_review"]')[0].text + "</p>"
    cursor.execute(insert_review, data)
    conn.commit()

In [19]:
# If it got a good score, we want to hightlight it"
if int(review.select('span[class="wineRatings_rating"]')[0].text) > 90:
    item_description = reviews[0].select('span[class="wineRatings_rating"]')[0].text + "pts. " + page_content.find(class_='pipWineNotes_copy viewMoreModule').text[:100]+"..."
else:
    item_description = page_content.find(class_='pipWineNotes_copy viewMoreModule').text[:112]+"..."

## Write to wine_master
We need to build out the Wine_master table, and this will be a new record for it. Eventually, we will have to keep this in sync with what is in the eCom.

## Write what little we can to Lightspeed
Lightspeed won't let you write directly to Product in eCom for things that are omni, so we have to make sure they are not active for eCom in Retail, then write using the retail api, then activate them for eCom.

In [33]:

data = {"msrp": msrp,
        "publishToEcom": True,
        "ItemECommerce":{
            "shortDescription":item_description,
            "longDescription": item_content,
            "weight": 1400        
            }
        }
lsr.update("Item", item_id, data)



In [43]:
data = {"Prices":{
            "ItemPrice":{
                "amount": 99.99,
                "useType":"MSRP",
                "useTypeID": 2
                }
}
        }
lsr.update("Item", 865, data)

# Do the Images
Gets the wine images off the page, catches all the information, stores it, and puts the image in Lightspeed via API

In [21]:
# images=page_content.select('body > div.corePage.winePage > main > section.pipMainContent > div.pipHero.prodImageZoom.js-thumbs-enabled > div.pipHero_wrapper > div > picture > img')
# images = page_content.get('img')
images = page_content.find_all(class_='pipThumb_image')
images

[<img alt="Ridge Pagani Ranch Zinfandel 2016 Front Label" class="pipThumb_image pipThumb_img-0" src="/product/images/w_75,h_75,c_pad,fl_progressive/mlmzi1ovp1hwnj6zaqtn.jpg"/>,
 <img alt="Ridge Pagani Ranch Zinfandel 2016  Front Bottle Shot" class="pipThumb_image pipThumb_img-1" src="/product/images/w_75,h_75,c_pad,fl_progressive/dtdjzop7jqszgwspquu8.jpg"/>]

In [22]:
for image in images:
    # urlparse will take the url from the image link...
    a = urlparse(image.get('src'))
    # Then we split out the filename and the extension
    (sourcefilename, ext) = os.path.splitext(os.path.basename(a.path))
    link = 'https://www.wine.com/product/images/'+ os.path.basename(a.path)
    ordering = "0"

    if "Bottle" in image.get('alt'):
        filename = system_id + '_btl'+ ext
        ordering = "1"
    elif "Label" in image.get('alt'):
        filename = system_id + '_lbl'+ ext
        ordering = "2"    
    else:
        filename = system_id + sourcefilename + ext
    process_image(link, image_path, filename, item_id, description = image.get('alt'), ordering = ordering)
    print(f"This is {image.get('alt')} from {a.path} saved as {filename}")
    

{"@attributes":{"count":"1"},"Image":{"imageID":"1114","description":"Ridge Pagani Ranch Zinfandel 2016 Front Label","filename":"210000000998_lbl.jpg","ordering":"2","publicID":"qsq2mkegibyt2hmfstln","baseImageURL":"https:\/\/res.cloudinary.com\/lightspeed-retail\/image\/upload\/","size":"65536","createTime":"2020-07-27T14:31:22+00:00","timeStamp":"2020-07-27T14:31:22+00:00","itemID":"994","itemMatrixID":"0"}}
This is Ridge Pagani Ranch Zinfandel 2016 Front Label from /product/images/w_75,h_75,c_pad,fl_progressive/mlmzi1ovp1hwnj6zaqtn.jpg saved as 210000000998_lbl.jpg
{"@attributes":{"count":"1"},"Image":{"imageID":"1115","description":"Ridge Pagani Ranch Zinfandel 2016  Front Bottle Shot","filename":"210000000998_btl.jpg","ordering":"1","publicID":"hpskerr5wjtg8eurudt2","baseImageURL":"https:\/\/res.cloudinary.com\/lightspeed-retail\/image\/upload\/","size":"229832","createTime":"2020-07-27T14:31:23+00:00","timeStamp":"2020-07-27T14:31:23+00:00","itemID":"994","itemMatrixID":"0"}}
Thi

In [23]:

url = 'https://api.lightspeedapp.com/API/Account/190211/Image.json'
files = {'image': ('210000001629_btl.jpg', open(image_path+'210000001629_btl.jpg', 'rb'), 'image/jpeg')}
payload = {'data': '{"description": "Ridge Pagani Ranch Zinfandel 2016 Front Label", "ordering": 2, "itemID": ' + item_id +'}'}

r = requests.post(url, files=files, data=payload, headers=lsr.headers)

print(r.text)

{"@attributes":{"count":"1"},"Image":{"imageID":"1116","description":"Ridge Pagani Ranch Zinfandel 2016 Front Label","filename":"210000001629_btl.jpg","ordering":"2","publicID":"yianeep5hdfn0lifrfgr","baseImageURL":"https:\/\/res.cloudinary.com\/lightspeed-retail\/image\/upload\/","size":"229832","createTime":"2020-07-27T14:31:24+00:00","timeStamp":"2020-07-27T14:31:24+00:00","itemID":"994","itemMatrixID":"0"}}


In [24]:
# This prints out the document in a formatted tree style and limits the number of characters returned
print(page_content.prettify()[0:2000])

<!DOCTYPE doctype html>
<html class="loading" data-view1="true" data-view1_10="true" data-view1_9="true" lang="en" xml:lang="en">
 <head>
  <meta content="Wine" name="salesChannel"/>
  <meta content="Zinfandel" name="productVarietal"/>
  <meta content="Sonoma Valley" name="productRegion"/>
  <meta content="$48.99" name="productPrice"/>
  <meta content="Sonoma Valley, Sonoma County, California" name="productOrigin"/>
  <meta content="510360" name="productID"/>
  <meta content="In Stock" name="ProductAvailability"/>
  <meta content="SOU913205_2016" name="pProductID"/>
  <meta content="Product Detail" name="pageType"/>
  <meta content="Ridge Pagani Ranch Zinfandel 2016 from Sonoma Valley, Sonoma County, California - Black cherry, pepper and chaparral on the nose. Briary red fruit, well defined, chalky tannins and refreshing acidity on the palate. Long sensuous finish. Since..." name="description"/>
  <meta content="Never" name="picked_membership"/>
  <meta content="true" name="CartLoaded"

In [25]:
page_content.select('body > div.lightBox > div.lightBoxContentWrap > span.lightBoxContent > img.lightBoxContent_image.lightBoxContent_image-active')

[]

In [26]:
images[1].get('class')[0]


'pipThumb_image'

In [27]:
a = urlparse(images[0].get('src'))
print(a.path)                    # Output: /kyle/09-09-201315-47-571378756077.jpg
print(os.path.basename(a.path))  # Output: 09-09-201315-47-571378756077.jpg

/product/images/w_75,h_75,c_pad,fl_progressive/mlmzi1ovp1hwnj6zaqtn.jpg
mlmzi1ovp1hwnj6zaqtn.jpg


In [28]:
link = images[0].attrs
link

{'alt': 'Ridge Pagani Ranch Zinfandel 2016 Front Label',
 'class': ['pipThumb_image', 'pipThumb_img-0'],
 'src': '/product/images/w_75,h_75,c_pad,fl_progressive/mlmzi1ovp1hwnj6zaqtn.jpg'}

# Put the images in Lightspeed Retail

# Descriptions
This is some of the Descriptions

In [29]:
page_content.select('body > div.corePage.winePage > main > section.pipSecContent > section.pipSecContent_topSection > div.pipWineNotes')

[<div class="pipWineNotes" data-view1_35="true">
 <h2 class="pipSecContent_headline">Winemaker Notes</h2>
 <div class="pipWineNotes_copy viewMoreModule" itemprop="description"><div class="viewMoreModule_text">Black cherry, pepper and chaparral on the nose. Briary red fruit, well defined, chalky tannins and refreshing acidity on the palate. Long sensuous finish. <p>Since 1991, Ridge has made zinfandel from the Pagani vineyard on the western side of Sonoma Valley. The majority of the vineyard was planted ninety to one hundred twenty years ago; portions of it were replanted between 2013—2018. The vineyard is meticulously tended by a quality-conscious family who has worked the land for four generations. An average site would have been abandoned during the thirteen years of Prohibition, or during the Depression. Its long survival is proof of its merit.</p><p>Blend: 88% Zinfandel, 9% Petite Sirah, 3% Alicante Bouschet</p></div><div class="viewMoreModule_link"> <a class="viewMoreWine" href="#

In [30]:
page_content.select("img[src^=http]")

[]

In [31]:
for image in images:
        lnk = link["src"]

